## Example of using `VaspRelaxationWorkChain`

In [2]:
%aiida demo

Loaded AiiDA DB environment - profile name: demo.

In [3]:
from pymatgen import get_structure_from_mp
from aiida.orm import StructureData
from aiida_user_addons.common.inputset.vaspsets import VASPInputSet
from aiida_user_addons.tools.dryrun import dryrun_vasp
from aiida_user_addons.vworkflows.relax import VaspRelaxWorkChain, RelaxOptions
from pprint import pprint

Just get a SrTiO3 structure from materials project

In [4]:
sto = StructureData(pymatgen=get_structure_from_mp("mp-5229"))

In [5]:
sto.get_pymatgen()

Structure Summary
Lattice
    abc : 3.94513 3.94513 3.94513
 angles : 90.0 90.0 90.0
 volume : 61.40220340476369
      A : 3.94513 0.0 0.0
      B : 0.0 3.94513 0.0
      C : 0.0 0.0 3.94513
PeriodicSite: Sr (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000]
PeriodicSite: Ti (1.9726, 1.9726, 1.9726) [0.5000, 0.5000, 0.5000]
PeriodicSite: O (1.9726, 0.0000, 1.9726) [0.5000, 0.0000, 0.5000]
PeriodicSite: O (1.9726, 1.9726, 0.0000) [0.5000, 0.5000, 0.0000]
PeriodicSite: O (0.0000, 1.9726, 1.9726) [0.0000, 0.5000, 0.5000]

Break the symmetry

In [8]:
sto.sites[0].x = 0.1

Load some default input dictionary, note one can customise it using the `overrides` settings.  
These sets are for convenience only - the full input is always recorded by AiiDA

In [12]:
inputset = VASPInputSet("UCLRelaxSet", sto, overrides={'encut': 400, 'lorbit': None})
inputset.get_input_dict()

{'prec': 'accurate',
 'encut': 400,
 'lreal': False,
 'ismear': 0,
 'sigma': 0.05,
 'ispin': 2,
 'algo': 'normal',
 'nwrite': 1,
 'icorelevel': 1,
 'lwave': False,
 'nelm': 200,
 'nelmin': 4,
 'lasph': True,
 'lvhar': True,
 'nedos': 2000,
 'isif': 3,
 'gga': 'ps',
 'ediff': 4.9999999999999996e-06,
 'ldauu': [0.0, 4.0, 0.0],
 'ldauj': [0.0, 0.0, 0.0],
 'ldautype': 2,
 'lmaxmix': 4,
 'ldaul': [-1, 2, -1],
 'ldau': True}

In [22]:
builder =VaspRelaxWorkChain.get_builder()
builder.structure = sto

# The actual calculations parameters are set under the `vasp` input port

builder.vasp.parameters = inputset.get_input_dict(raw_python=False)
builder.vasp.potential_family = Str('PBE.54')
builder.vasp.potential_mapping = Dict(dict=inputset.get_pp_mapping())
builder.vasp.options = Dict(dict={
    'resources': {'num_machines': 1, 'tot_num_mpiprocs': 4}
})
builder.vasp.kpoints_spacing = Float(0.07)   # Use a coarse kpoint spacing of 0.07 A^-1 2*pi
builder.vasp.code = Code.get_from_string("vasp@localhost")
builder.metadata.label = 'STO broken'

Set the relaxation settings  
The `RelaxOptions` provides some default parameters to control the relaxation

In [23]:
relax_options = RelaxOptions(convergence_on=True, 
                             # Converge between the input and output structure, need when we expect large cell colume change to ensure basis set consistency
                             # The default is 'last', which will check against the last two structures
                             convergence_mode='inout'   
                            )

In [24]:
pprint(relax_options.to_dict())

{'algo': 'cg',
 'convergence_absolute': False,
 'convergence_max_iterations': 5,
 'convergence_mode': 'inout',
 'convergence_on': True,
 'convergence_positions': 0.1,
 'convergence_shape_angles': 0.1,
 'convergence_shape_lengths': 0.1,
 'convergence_volume': 0.01,
 'force_cutoff': 0.03,
 'perform': True,
 'positions': True,
 'shape': True,
 'steps': 60,
 'volume': True}


One can also set them interactively, just like the `builder`

In [25]:
relax_options.force_cutoff = 0.05  ## Try tab-completion with other keys

In [26]:
builder.relax_settings = relax_options.to_aiida_dict()

Finally we submit the calculation

In [27]:
from aiida.engine import submit

In [28]:
running = submit(builder)
running

<WorkChainNode: uuid: b600f44c-3b54-44c0-8ec4-06cc7a967e3d (pk: 34559) (aiida.workflows:vaspu.relax)>

## Analysis

load the node using UUID - this ensures that if we restart the notebook we can carry on with the analysis

In [29]:
work = load_node("b600f44c")  

Verbose report emitted while the procss was running - we can see that the workchain checked the convergence and resubmit the job if not happy.  
It will also do a final calculation to ensure the energy is accurate - VASP can only do constant-basis relaxations. 

In [32]:
!verdi process report b600f44c

2020-09-27 20:09:47 [5104 | REPORT]: [34559|VaspRelaxWorkChain|run_relax]: launching VaspWorkChain<34562> iterations #1
2020-09-27 20:09:48 [5105 | REPORT]:   [34562|VaspWorkChain|run_calculation]: launching VaspCalculation<34565> iteration #1
2020-09-27 20:11:03 [5106 | REPORT]:   [34562|VaspWorkChain|_handle_succesfull]: VaspCalculation<34565> completed successfully
2020-09-27 20:11:03 [5107 | REPORT]:   [34562|VaspWorkChain|results]: VaspWorkChain<34562> completed after 1 iterations
2020-09-27 20:11:03 [5108 | REPORT]:   [34562|VaspWorkChain|on_terminated]: cleaned remote folders of calculations: 34565
2020-09-27 20:11:03 [5109 | REPORT]: [34559|VaspRelaxWorkChain|analyze_convergence]: Checking the convergence of the relaxation.
2020-09-27 20:11:03 [5110 | REPORT]: [34559|VaspRelaxWorkChain|check_positions_convergence]: max site position change is 0.03543, tolerance is 0.1 - OK
2020-09-27 20:11:03 [5111 | REPORT]: [34559|VaspRelaxWorkChain|check_volume_convergence]: cell volume chan

In [34]:
work.inputs.structure.get_cell_volume()

61.40220340476369

In [35]:
work.outputs.relax__structure.get_cell_volume()

59.1958127334041

In [40]:
work.outputs.misc.get_dict()

{'symmetries': {'point_group': {'static': [None], 'dynamic': [None]},
  'primitive_translations': [1],
  'num_space_group_operations': {'static': [48], 'dynamic': [48]}},
 'stdout_error': [{'message': 'WARNING: Linux kernel CMA support was requested via the',
   'critical': False,
   'shortname': 'general'}],
 'magnetization': [-0.0001913],
 'maximum_force': 0.0,
 'maximum_stress': 0.07710148,
 'total_energies': {'energy_no_entropy': -38.92753782}}

## Provenance graph example

In [36]:
from aiida.tools.visualization import Graph

g = Graph()
g.recurse_descendants(work)
g.recurse_ancestors(work, depth=1)

In [38]:
g.graphviz.render('relax', format='png')

'relax.png'

![Relaxation Provenance](relax.png)